In [1]:
from dataclasses import dataclass
import itertools
import math
import re
import typing

In [2]:
def res(day, level, result):
    print(f'Result of [Day {day}, Level {level}]: {result}')

In [3]:
with open('../data/07_input.txt') as f:
    input = f.read()

In [4]:
# all methods assume no loops
class Bag:
    overall_regex = re.compile(r"(.*) bags contain (?:no other bags|(.*)).")
    ref_regex = re.compile(r"(\d+) (.*) (?:bag|bags)")

    def __init__(self, name, pointsMap):
        self.name : str = name
        self.pointsMap : Map[str, int] = pointsMap
        self.points = [x for name, count in self.pointsMap.items() for x in [name]*count]
        self.pointedMap : Map[str, int] = {}

    @classmethod
    def fromString(cls, string):
        name, containsPart = cls.overall_regex.search(string).groups()
        if(containsPart):
            points = {match.group(2) : int(match.group(1)) 
            for match in [cls.ref_regex.search(part) for part in containsPart.split(", ")]}
        else:
            points = dict() 
        return cls(name, points)

    def ancestors(self, bag_map):
        if(not self.pointedMap):
            return set()
        else:
            recursion = [bag_map[key].ancestors(bag_map) for key in self.pointedMap.keys()]
            return set().union(*recursion, self.pointedMap.keys())  

    def descendentsMultiple(self, bag_map):
        if(not self.points):
            return []
        else:
            recursion = [x for key in self.points for x in bag_map[key].descendentsMultiple(bag_map)]
            return recursion + self.points
    
    def __str__(self):
        return f'{self.name}\n  ---> {self.to}\n  <--- {self.pointed}'
    def __repr__(self):
        return self.__str__()

In [5]:
data = [Bag.fromString(x) for x in input.split('\n') if len(x) > 0]
bag_map = {bag.name : bag for bag in data}
for bag in data:
    for (to_name, to_count) in bag.pointsMap.items():
        bag_map[to_name].pointedMap[bag.name] = to_count

In [6]:
shiny_gold_bag = bag_map['shiny gold']
ancestors = shiny_gold_bag.ancestors(bag_map)

res(7, 1, len(ancestors))

Result of [Day 7, Level 1]: 372


In [7]:
shiny_gold_bag = bag_map['shiny gold']
descendants = shiny_gold_bag.descendentsMultiple(bag_map)

res(7, 2, len(descendants))

Result of [Day 7, Level 2]: 8015
